In [ ]:
# default_exp model

# Models

> API details.

In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib as mpl
%matplotlib inline

In [ ]:
#export
import warnings

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models

import pytorch_lightning as pl
from pytorch_lightning.core import LightningModule
from pytorch_lightning.metrics import functional as FM

C:\Users\bomco\miniconda3\envs\devtorch\lib\site-packages\pytorch_lightning\utilities\distributed.py:23: UserWarning: Unsupported `ReduceOp` for distributed computing.
  warnings.warn(*args, **kwargs)


In [ ]:
#export
from isic.dataset import SkinDataModule
from isic.layers import LabelSmoothingCrossEntropy, LinBnDrop, AdaptiveConcatPool2d
from isic.callback.hyperlogger import HyperparamsLogger
from isic.callback.logtable import LogTableMetricsCallback
from isic.callback.mixup import MixupDict
from isic.callback.cutmix import CutmixDict
from isic.utils import reduce_loss, apply_init

In [ ]:
#export
class ResnetModel(LightningModule):
    def __init__(self, steps_epoch, epochs=30, lr=1e-2, wd=0.9, n_out=7, concat_pool=True):
        super().__init__()
        self.save_hyperparameters()
        mdl = models.resnet50(pretrained=True)
        # create body
        cut = -2
        
        body = nn.Sequential(*list(mdl.children())[:cut])
        # create head
        num_ftrs = mdl.fc.in_features * (2 if concat_pool else 1)
        lin_ftrs = [num_ftrs, 512, n_out]
        p = 0.5
        ps = [p/2] * (len(lin_ftrs) - 2) + [p]
        actns = [nn.ReLU(inplace=True)] * (len(lin_ftrs) - 2) + [None]
        pool = AdaptiveConcatPool2d() if concat_pool else nn.AdaptiveAvgPool2d(1)
        layers = [pool, nn.Flatten()]
        for ni, no, p, actn in zip(lin_ftrs[:-1], lin_ftrs[1:], ps, actns):
            layers += LinBnDrop(ni, no, bn=True, p=p, act=actn)
        
        head = nn.Sequential(*layers)
        #model
        self.model = nn.Sequential(body, head)
        apply_init(self.model[1])
        
        self.loss_func = nn.CrossEntropyLoss()

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch['img'], batch['label']
        y_hat = self(x)
        loss = self.loss_func(y_hat, y)
        acc = FM.accuracy(y_hat, y, num_classes=7)
        result = pl.TrainResult(minimize=loss)
        result.log('train_loss', loss)
        result.log('train_acc', acc, prog_bar=True)
        return result

    def validation_step(self, batch, batch_idx):
        x, y = batch['img'], batch['label']
        y_hat = self(x)
        loss = self.loss_func(y_hat, y)
        acc = FM.accuracy(y_hat, y, num_classes=7)
        result = pl.EvalResult(checkpoint_on=loss)
        result.log('val_loss', loss, prog_bar=True) 
        result.log('val_acc', acc, prog_bar=True)
        return result

    def configure_optimizers(self):
        opt = torch.optim.Adam(self.parameters(), lr=1e-2, weight_decay=self.hparams.wd)
        scheduler = torch.optim.lr_scheduler.OneCycleLR(opt, max_lr=self.hparams.lr, steps_per_epoch=self.hparams.steps_epoch, epochs=self.hparams.epochs)
        sched = {
            'scheduler': scheduler, # The LR schduler
            'interval': 'step', # The unit of the scheduler's step size
            'frequency': 1, # The frequency of the scheduler
            'reduce_on_plateau': False, # For ReduceLROnPlateau scheduler
        }
        return [opt], [sched]

In [ ]:
message_formater = "You have set {0} number of classes if different from predicted {0} and target {0} number of classes"
warnings.filterwarnings("ignore", message_formater.format("(.*)"), category=UserWarning)

In [ ]:
dm = SkinDataModule()
dm.prepare_data()
dm.setup('fit')

In [ ]:
# init model
model = ResnetModel(steps_epoch=140, epochs=3, lr=1e-2)
hp_log = HyperparamsLogger()

# most basic trainer, uses good defaults
trainer = pl.Trainer(max_epochs=3, callbacks=[LogTableMetricsCallback(), hp_log, CutmixDict()], fast_dev_run=True)

Running in fast_dev_run mode: will run a full train, val and test loop using a single batch
GPU available: False, used: False
TPU available: False, using: 0 TPU cores


In [ ]:
model

ResnetModel(
  (model): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (4): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
          (

In [ ]:
trainer.fit(model, dm)


  | Name      | Type             | Params
-----------------------------------------------
0 | model     | Sequential       | 25 M  
1 | loss_func | CrossEntropyLoss | 0     


C:\Users\bomco\miniconda3\envs\devtorch\lib\site-packages\pytorch_lightning\utilities\distributed.py:23: UserWarning: 
                    When using EvalResult(early_stop_on=X) or TrainResult(early_stop_on=X) the
                    'monitor' key of ModelCheckpoint has no effect.
                    Remove ModelCheckpoint(monitor='loss) to fix')
                
  warnings.warn(*args, **kwargs)


epoch,train_loss,train_acc,val_loss,val_acc
1,3.306,0.078125,1.312354,0.468750


1

In [ ]:
%reload_ext tensorboard

In [ ]:
%tensorboard --logdir=lightning_logs/

Reusing TensorBoard on port 6006 (pid 4636), started 1 day, 1:44:30 ago. (Use '!kill 4636' to kill it.)

In [ ]:
!tensorboard --logdir=lightning_logs/

In [ ]:
# #
# class ResnetModel(LightningModule):
#     def __init__(self):
#         super().__init__()
# #         self.save_hyperparameters()
#         self.resnet = models.resnet50(pretrained=True)
#         num_ftrs = self.resnet.fc.in_features
#         self.resnet.fc = nn.Linear(num_ftrs, 7)
#         self.loss_func = F.cross_entropy

#     def forward(self, x):
#         return self.resnet(x)

#     def training_step(self, batch, batch_idx):
#         print(batch)
#         if hasattr(self, 'enhanced_batch'):
#             print('hehe')
#             batch = self.enhanced_batch
#         x, y = batch
#         y_hat = self(x)
#         loss = self.loss_func(y_hat, y)
#         acc = FM.accuracy(y_hat, y, num_classes=7)
#         result = pl.TrainResult(minimize=loss)
#         result.log('train_loss', loss)
#         result.log('train_acc', acc, prog_bar=True)
#         return result

#     def validation_step(self, batch, batch_idx):
#         x, y = batch
#         y_hat = self(x)
#         loss = F.cross_entropy(y_hat, y)
#         acc = FM.accuracy(y_hat, y, num_classes=7)
#         result = pl.EvalResult(checkpoint_on=loss)
#         result.log('val_loss', loss, prog_bar=True) 
#         result.log('val_acc', acc, prog_bar=True)
#         return result

#     def configure_optimizers(self):
#         opt = torch.optim.Adam(self.parameters(), lr=1e-2)
#         scheduler = torch.optim.lr_scheduler.OneCycleLR(opt, max_lr=1e-2, steps_per_epoch=140, epochs=10)
#         sched = {
#             'scheduler': scheduler, # The LR schduler
#             'interval': 'step', # The unit of the scheduler's step size
#             'frequency': 1, # The frequency of the scheduler
#             'reduce_on_plateau': False, # For ReduceLROnPlateau scheduler
#         }
#         return [opt], [sched]

In [ ]:
from nbdev.export import *
notebook2script('model.ipynb')

Converted cb_mixup.ipynb.
